The goal is for the model to take:

1) Time
2) Transmit Grid Square

Output:
1) Most likely Grid Squares to contact
2) Which band to use
3) Which Mode to use

Label is: db

In [25]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import pandas as pd

In [39]:
class DXDataset(Dataset):
    def __init__(self, file, transform=None, target_transform=None):
        self.data = pd.read_csv(file)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data = self.data.iloc[idx].drop("db")
        label = self.data.iloc[idx]["db"]
        if self.transform:
            data = self.transform(data)
        if self.target_transform:
            label = self.target_transform(label)
        return data, label

In [40]:
training_data = DXDataset("data/train.csv")
test_data = DXDataset("data/test.csv")

training_data.__getitem__(1)[0]

Unnamed: 0                                             1
de_pfx                                                DL
freq                                              3521.0
band                                                 80m
dx_pfx                                                DL
mode                                                  CQ
date                                 2022-01-01 00:00:00
speed                                               21.0
tx_mode                                               CW
Beijing_A_index                                     -1.0
Beijing_K_index                                     -1.0
Belsk_A_index                                       -1.0
Belsk_K_index                                       -1.0
Boulder_A_index                                     11.0
Boulder_K_index                                     11.0
Cape Chelyuskin_A_index                             -1.0
Cape Chelyuskin_K_index                             -1.0
Chambon-la-foret_A_index       

In [ ]:
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [ ]:
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
input = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(input, cmap="gray")
plt.show()
print(f"Label: {label}")

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found object

In [44]:
import torch
import torch.nn.functional as F

x = torch.tensor([4, 1])
F.one_hot(x, num_classes=10)

tensor([[0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]])

In [47]:
import torch
from torchtext.vocab import vocab
from torchtext.transforms import VocabTransform
from collections import OrderedDict
vocab_obj = vocab(OrderedDict([('a', 1), ('b', 1), ('c', 1)]))
vocab_obj

Vocab()

In [48]:
vocab_transform = VocabTransform(vocab_obj)
vocab_transform

VocabTransform(
  (vocab): Vocab()
)

In [49]:
output = vocab_transform([['a','b'],['a','b','c']])
output

[[0, 1], [0, 1, 2]]

In [50]:
jit_vocab_transform = torch.jit.script(vocab_transform)
jit_vocab_transform

RecursiveScriptModule(
  original_name=VocabTransform
  (vocab): RecursiveScriptModule(original_name=Vocab)
)